# RED NEURONAL PREDICTORA

En este notebook se podrá visualizar el codigo relacionado con el aprendizaje no supervisado unicamente. Para visualizar el codigo relacionado con el aprendizaje supervisado acceder al archivo de este mismo repositorio: red_neuronal_predictora.ipynb (para la red neuronal predictora), arbol_decisión.ipynb (para el arbol de decisión) o red_neuronal_hiperparametros.ipynb (para la el ajuste de hiperparametros)

## Importación de las librerías 

A continuación se importan las librerias necesarias. Adicionalmente se configuran unas variables de entorno para obtener los mismos resultados para las diferentes ejecuciones. 


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)



import os
os.environ['PYTHONHASHSEED'] = '42'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

## Cargar los datos

El primer paso es cargar el dataset adecuadamente. Recordemos que el archivo csv esta preprocesado con antelación y al estar en el aprendizaje no supervisado las columnas "ID Usuario" y la etiqueta "Conclusion" se prescinde de ellas. 

In [ ]:
# Cargar los datos desde el archivo CSV
file_path = 'supervisado_final.csv'  # Archivo CSV
data = pd.read_csv(file_path, sep=';', decimal='.')

# Seleccionar todas las columnas excepto la primera ('ID Usuario') y la última (la etiqueta)
features = data.iloc[:, 1:-1]

## Clustering

En este apartado se podrán visualizar los diferentes resultados obtenidos para un numero de cluster modificable, el actual K = 3, el que mejores resultados ofrece trás el análisis.

Librerias especificas

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# Número de clusters, K
num_clusters = 3

# Aplicar K-means con k=3 y calcular la métrica Silhouette
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(features)

Obtención de la metrica Silhouette:

In [ ]:
silhouette_avg = silhouette_score(features, clusters)
print(f'\n\n\nSilhouette score for {num_clusters} clusters: {silhouette_avg}')

Añadir al dataset los clusters asignados e imprimir la cantidad de trabajadores por cluster.

In [ ]:
# Añadir el cluster asignado a cada observación
data['Cluster'] = clusters

# Contar el número de ítems en cada cluster
cluster_counts = data['Cluster'].value_counts().sort_index()

# Imprimir el número de ítems en cada cluster
print("\n\n\nNúmero de ítems en cada cluster:")
print(cluster_counts)


A continuación se calcula la media de cada variable/columna para todos los clusers.

In [ ]:
# Calcular la media de cada columna en cada cluster
cluster_means = data.groupby('Cluster').mean()

# Eliminar las columnas de ID Usuario y de Conclusion
cluster_means = cluster_means.drop(['ID Usuario', 'Conclusion'], axis=1)

# Imprimir la media de cada columna en cada cluster
print("\n\n\nMedia de cada columna en cada cluster:")
print(cluster_means)

Calcular el ranking de la "separabilidad de variables".

In [ ]:
# Generar todas las combinaciones de pares de clusters
cluster_pairs = list(combinations(range(num_clusters), 2))

# Calcular la diferencia total entre clusters para cada variable de forma genérica
total_diff = sum(np.abs(cluster_means.loc[pair[0]] - cluster_means.loc[pair[1]]) for pair in cluster_pairs) / len(cluster_pairs)

# Ordenar las variables según la diferencia total entre clusters de forma descendente
ordered_columns = total_diff.sort_values(ascending=False)

Mostrar tanto en consola como en un diagrama de barras los resultados

In [ ]:
# Imprimir las columnas ordenadas
print("\n\n\nColumnas ordenadas según la diferencia total entre clusters:")
print(ordered_columns)

# Crear el diagrama de barras
plt.figure(figsize=(10, 8))
plt.bar(ordered_columns.index, ordered_columns.values)

# Añadir etiquetas y título
plt.xlabel('Variables')
plt.ylabel('Diferencia total entre clusters')
plt.title('Variables ordenadas según la diferencia total entre clusters')

# Rotar las etiquetas del eje x para que sean legibles
plt.xticks(rotation=90)

# Mostrar el diagrama de barras
plt.show()

Seleccionamos las variables que superen el umbral 0.24 siendo este umbral la linea de división analizada. Imprimimos las medias de esas variables seleccionadas para todos los clusters para una mayor legibilidad y posterior comparación.

In [ ]:
# Seleccionar solo las columnas que superen un umbral de 0.24
selected_columns = ordered_columns[ordered_columns > 0.24]

# Imprimir las columnas seleccionadas
print("\n\n\nColumnas seleccionadas:")
print(selected_columns)

# Seleccionar solo las filas correspondientes en el DataFrame cluster_means
selected_means = cluster_means.loc[:, selected_columns.index]

# Imprimir las medias seleccionadas
print("\n\n\nMedias seleccionadas:")
print(selected_means)

## Reducción de Dimensionalidad

En este apartado se obserban dos diferentes metodos de reducción de dimendsionalidad y diferentes caracteristicas de cada una de ellas para la selección de la mejor adaptada al problema.

Librerias especificas

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors

### PCA

Obtención de los componentes principales para representar los resultados en 2D

In [ ]:
# Aplicar PCA para obtener las componentes principales
pca = PCA(n_components=2)
pca_components = pca.fit_transform(features)

Imprimir la varianza de las dos primeras componentes principales para saber la información que mantiene.

In [ ]:
# Varianza explicada por las dos primeras componentes principales
explained_variance_ratio = pca.explained_variance_ratio_
print(f"\n\n\nVarianza explicada por la PC1: {explained_variance_ratio[0]:.4f}")
print(f"\nVarianza explicada por la PC2: {explained_variance_ratio[1]:.4f}")
print(f"\nVarianza total explicada por las dos primeras componentes principales: {explained_variance_ratio[:2].sum():.4f}")

Obtención de los componentes principales ordenados por valor absoluto para saber que caracteristicas han sido seleccionadas para la representación en 2D

In [ ]:
# Cargar las componentes principales ordenadas por valor absoluto
loadings = pd.DataFrame(pca.components_.T, columns=['PCA1', 'PCA2'], index=features.columns)
sorted_loadings = loadings.abs().sort_values(by=['PCA1', 'PCA2'], ascending=False)
print("\n\n\nCargas de los componentes principales ordenadas por valor absoluto:")
print(sorted_loadings)

### t-SNE



In [ ]:
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(features)

Evaluación de la reducción de dimensionalidad para posterior comparación con la varianza obtenida por las dos primeras PCA. Se hace un KNN = 5 (probar con diferentes valores) tanto para los datos originales como para los reducidos por t-SNE y se comparan para saber la consistencia de vecindad. 

In [ ]:
# Evaluar la preservación de vecindad
n_neighbors = 5
nn_original = NearestNeighbors(n_neighbors=n_neighbors).fit(features)
_, neighbors_original = nn_original.kneighbors(features)
nn_tsne = NearestNeighbors(n_neighbors=n_neighbors).fit(tsne_results)
_, neighbors_tsne = nn_tsne.kneighbors(tsne_results)

# Calcular la consistencia de vecindad
consistencies = [len(set(neighbors_original[i]).intersection(set(neighbors_tsne[i]))) / n_neighbors for i in range(features.shape[0])]
mean_consistency = np.mean(consistencies)
print(f'\n\n\nConsistencia de vecindad media: {mean_consistency:.4f}')

Visualización de los clusters utilizando t-SNE.

In [ ]:
# Visualizar los clusters utilizando t-SNE
plt.figure(figsize=(10, 8))
sns.scatterplot(x=tsne_results[:, 0], y=tsne_results[:, 1], hue=clusters, palette='viridis')
plt.title('Clusters visualizados usando t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Cluster')
plt.show()

### Detección de anomalias